In [1]:
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
#import pandas as pd
import os,sys
import math
import os.path
import vtk
from vtk.numpy_interface import dataset_adapter as dsa
from vtk.numpy_interface import algorithms as algs
from vtk.util.numpy_support import vtk_to_numpy
import sklearn
from sklearn.naive_bayes import MultinomialNB
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from time import time
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from vtk import vtkImageImport
from vtk.util.vtkConstants import *
import cv2
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import Conv3D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.utils import np_utils
from keras.models import load_model
from skimage.transform import resize
from skimage import io, transform
import pickle
from openpyxl import load_workbook
import xlrd

KeyboardInterrupt: 

In [ ]:
from itertools import combinations_with_replacement
import numpy as np
from scipy import ndimage as ndi
#from torch import absolute_eigenvaluesh
from scipy import absolute

In [ ]:
def compute_3d_hessian_matrix(nd_array, sigma=1, scale=True, whiteonblack=True):
    """
    Computes the hessian matrix for an nd_array.
    This can be used to detect vesselness as well as other features.
    In 3D the first derivative will contain three directional gradients at each index:
    [ gx,  gy,  gz ]
    The Hessian matrix at each index will then be equal to the second derivative:
    [ gxx, gxy, gxz]
    [ gyx, gyy, gyz]
    [ gzx, gzy, gzz]
    The Hessian matrix is symmetrical, so gyx == gxy, gzx == gxz, and gyz == gzy.
    :param nd_array: n-dimensional array from which to compute the hessian matrix.
    :param sigma: gaussian smoothing to perform on the array.
    :param scale: if True, the hessian elements will be scaled by sigma squared.
    :return: hessian array of shape (..., ndim, ndim)
    """
    ndim = nd_array.ndim

    # smooth the nd_array
    smoothed = ndi.gaussian_filter(nd_array, sigma=sigma, mode='nearest', truncate=3.0)

    # compute the first order gradients
    gradient_list = np.gradient(smoothed)

    # compute the hessian elements
    hessian_elements = [np.gradient(gradient_list[ax0], axis=ax1)
                        for ax0, ax1 in combinations_with_replacement(range(ndim), 2)]

    if sigma > 0 and scale:
        # scale the elements of the hessian matrix
        if whiteonblack:
            hessian_elements = [(sigma ** 2) * element for element in hessian_elements]
        else:
            hessian_elements = [-1 * (sigma ** 2) * element for element in hessian_elements]

    # create hessian matrix from hessian elements
    hessian_full=[[()] * ndim for x in range(ndim)]
    #hessian_full = [[None] * ndim] * ndim

    for index, (ax0, ax1) in enumerate(combinations_with_replacement(range(ndim), 2)):
        element = hessian_elements[index]
        hessian_full[ax0][ax1] = element
        if ax0 != ax1:
            hessian_full[ax1][ax0] = element

    hessian_rows = list()
    for row in hessian_full:
        #print(row.shape)
        hessian_rows.append(np.stack(row, axis=-1))

    hessian = np.stack(hessian_rows, axis=-2)
    return hessian


def absolute_3d_hessian_eigenvalues(nd_array, sigma=1, scale=True, whiteonblack=True):
    """
    Eigenvalues of the hessian matrix calculated from the input array sorted by absolute value.
    :param nd_array: input array from which to calculate hessian eigenvalues.
    :param sigma: gaussian smoothing parameter.
    :param scale: if True hessian values will be scaled according to sigma squared.
    :return: list of eigenvalues [eigenvalue1, eigenvalue2, ...]
    """
    return np.absolute(compute_3d_hessian_matrix(nd_array, sigma=sigma, scale=scale, whiteonblack=whiteonblack))

In [ ]:
d3_plat_1 = [[0,0,0],[-1,-1,0],[0,-1,0],[1,-1,0],[1,0,0],[1,1,0],[0,1,0],[-1,1,0],[-1,0,0],
             [0,0,1],[-1,-1,1],[0,-1,1],[1,-1,1],[1,0,1],[1,1,1],[0,1,1],[-1,1,1],[-1,0,1],
             [0,0,-1],[-1,-1,-1],[0,-1,-1],[1,-1,-1],[1,0,-1],[1,1,-1],[0,1,-1],[-1,1,-1],[-1,0,-1]]
d2_plat_1 = [[0,0,0],[-1,-1,0],[0,-1,0],[1,-1,0],[1,0,0],[1,1,0],[0,1,0],[-1,1,0],[-1,0,0]]
d3_plat_1_5 = [[0,0,0],[-5,-5,0],[0,-5,0],[5,-5,0],[5,0,0],[5,5,0],[0,5,0],[-5,5,0],[-5,0,0],
               [0,0,5],[-5,-5,5],[0,-5,5],[5,-5,5],[5,0,5],[5,5,5],[0,5,5],[-5,5,5],[-5,0,5],
               [0,0,-5],[-5,-5,-5],[0,-5,-5],[5,-5,-5],[5,0,-5],[5,5,-5],[0,5,-5],[-5,5,-5],[-5,0,-5]]



In [ ]:
def read_data(path):#定义一个“读取图像”函数，用来读取图像
    image_data = nib.load(path)
    return image_data

In [ ]:
path_1 = r'E:\article3\样例文件\Branches\P60\P60_same1deno_com2.nii'#图像路径，日后可以根据需要修改
target_1 = read_data(path_1)#使用“读取图像”函数

target_3d_1 = target_1.get_fdata()
target_3d_1.shape

In [ ]:
path_2 = r'E:\article3\样例文件\Branches\P60\P60_insert1.nii'#图像路径，日后可以根据需要修改
target_2 = read_data(path_2)#使用“读取图像”函数

target_3d_insert2 = target_2.get_fdata()

In [ ]:
target_3d_normalization = (target_3d_1/1000)*255

In [ ]:
def predict(origin):
    aa,bb,cc = origin.shape
    target = np.zeros((aa-10,bb-10,cc-10))
    a,b,c = target.shape
    #a = aa-10
    #b = bb-10
    #c = cc-10
    
    #model_artifact = load_model(r'E:\article3\样例文件\Branches\S57\micro_artifact1.h5')
    #model_micro = load_model(r'E:\article3\样例文件\Branches\S57\micro_vessel.h5')
    model_complex = load_model(modelpath)

    #gc.disable()
    semble = []
    for x in range(a):
        #print(x)
        for y in range(b):
            for z in range(c):
                semble.append([x,y,z])
    
    #result = np.zeros((len(semble)))
    for n in range(len(semble)):
        if n%1000000 == 0:
            print(n/1000000)
        x,y,z = semble[n]
        if origin[x+5,y+5,z+5] < 45:
            target[x,y,z] = 0
            #result[n] = 0
        elif origin[x+5,y+5,z+5] > 100:
            target[x,y,z] = 255
            #result[n] = 255
        else:
            #非管桩伪影检测
            #region_artifact = np.zeros((11,11,11))
            #for i in range(11):
            #    for j in range(11):
            #        for k in range(11):
            #            region_artifact[i,j,k] = origin[x+7-5+i,y+7-5+j,z+5-5+k]
            #img_artifact = resize(region_artifact, output_shape=(1,11,11,11,1))
            #model_artifact = load_model(r'E:\article3\样例文件\Branches\S57\micro_artifact.h5')
            #prediction_artifact = model_artifact.predict_classes(img_artifact)

            #小血管检测
            #region_micro = np.zeros((7,7,7))
            #for i in range(7):
            #    for j in range(7):
            #        for k in range(7):
            #            region_micro[i,j,k] = origin[x+7-3+i,y+7-3+j,z+5-3+k]
            #img_micro = resize(region_micro, output_shape=(1,7,7,7,1))
            #model_micro = load_model(r'E:\article3\样例文件\Branches\S57\micro_vessel.h5')
            #prediction_micro = model_micro.predict_classes(img_micro)
            
            #复合检测
            region_complex = np.zeros((11,11,11))
            for i in range(11):
                for j in range(11):
                    for k in range(11):
                        region_complex[i,j,k] = origin[x+5-5+i,y+5-5+j,z+5-5+k]
            img_complex = resize(region_complex, output_shape=(1,11,11,11,1))
            #model_micro = load_model(r'E:\article3\样例文件\Branches\S57\micro_vessel.h5')
            #prediction_complex = model_complex.predict(img_complex)#predict_classes(img_complex)
            prediction = model_complex(img_complex)
            #print(prediction_complex)
            #print([x,y,z],target[x,y,z])
            if prediction[0][1] >= 0.5:
                target[x,y,z] = 255
                #result[n] = 255
            else:
                target[x,y,z] = 0
                #result[n] = 0
    
    #npresult = np.array(result)
    #target_result = result.reshpae((512,640,248))

    #gc.enable()
    
    #mask1,mask2,mask3,mask4 = np.vsplit(target,4)
                        
    return target

In [ ]:
target_3d_predict = predict(target_3d_normalization)

In [ ]:
target_3d_insert_clear_1 = target_3d_insert_clear

In [ ]:
spline_points = [[[178, 231, 159], [155, 241, 116], [147, 245, 101], [147, 247, 97], [154, 249, 90], [154, 258, 78], [155, 266, 71], [153, 272, 67], [151, 279, 63], [150, 284, 60], [150, 285, 60]], [[229, 188, 161], [270, 168, 171], [286, 161, 175], [288, 156, 175], [293, 146, 177], [296, 144, 177], [306, 138, 182], [317, 131, 188], [325, 133, 194], [323, 131, 193], [327, 131, 195], [330, 132, 196], [335, 136, 195], [349, 145, 194], [351, 146, 194], [350, 144, 194], [349, 144, 194], [351, 145, 191], [359, 150, 187], [366, 155, 182], [369, 159, 181], [371, 164, 177], [374, 169, 175], [378, 175, 173], [379, 184, 171]], [[298, 187, 101], [292, 187, 108], [286, 187, 117], [274, 184, 125], [255, 177, 138], [247, 177, 145], [237, 185, 154], [228, 190, 160], [230, 185, 162], [226, 190, 164], [219, 199, 166], [212, 204, 167], [205, 209, 167], [198, 216, 165], [190, 223, 163], [181, 228, 160], [172, 234, 156], [165, 236, 153], [147, 241, 141], [127, 247, 130], [116, 253, 130], [108, 257, 126], [107, 257, 126]], [[108, 257, 126], [108, 246, 142], [118, 236, 152], [120, 234, 153], [127, 229, 154], [135, 224, 154], [142, 219, 155], [148, 213, 158], [155, 208, 159], [167, 194, 167], [168, 187, 173], [170, 179, 179], [170, 171, 185], [166, 165, 194], [161, 161, 201], [155, 156, 209], [145, 150, 214], [137, 145, 219], [128, 139, 223], [120, 133, 227], [114, 125, 232], [109, 116, 238], [107, 106, 245], [109, 98, 255], [111, 91, 262], [113, 86, 270], [112, 84, 280]], [[113, 254, 129], [117, 252, 145], [124, 241, 156], [129, 239, 157], [135, 229, 157], [146, 225, 171], [153, 217, 174], [162, 213, 177], [169, 206, 177], [177, 200, 179], [184, 195, 179], [192, 191, 180], [198, 185, 182], [205, 182, 187], [212, 178, 190], [219, 171, 190], [223, 162, 193], [217, 148, 193], [203, 149, 196], [193, 157, 198], [186, 163, 205], [178, 167, 211], [169, 173, 215], [158, 175, 218], [147, 175, 220], [137, 175, 226], [129, 174, 235], [126, 170, 247], [127, 172, 259], [129, 169, 268], [139, 163, 273], [148, 167, 279]], [[247, 177, 145], [247, 177, 132], [261, 164, 117], [273, 153, 116], [279, 149, 118], [283, 145, 124], [287, 143, 132], [289, 140, 141], [290, 136, 149], [291, 132, 159], [291, 128, 169], [289, 122, 176], [287, 114, 183], [283, 105, 188], [279, 95, 191], [273, 85, 191], [269, 73, 191], [265, 62, 191], [263, 51, 190], [259, 40, 188], [248, 32, 190], [235, 29, 196], [224, 30, 202], [214, 30, 207], [210, 30, 208]], [[110, 96, 257], [115, 87, 267], [121, 83, 274], [125, 80, 280], [128, 78, 285], [131, 79, 294], [134, 81, 303], [134, 84, 312], [141, 88, 324], [149, 91, 331], [151, 92, 333]], [[667, 207, 159], [682, 231, 147], [690, 242, 143], [690, 243, 142]], [[562, 178, 174], [527, 195, 194], [514, 197, 191], [499, 197, 185], [486, 203, 176], [479, 205, 171], [465, 213, 162], [460, 216, 155], [415, 235, 138], [404, 239, 142], [392, 246, 153], [391, 246, 155]], [[505, 222, 103], [514, 204, 121], [520, 194, 132], [524, 187, 138], [532, 180, 144], [541, 174, 147], [553, 177, 163], [560, 179, 175], [564, 175, 172], [571, 175, 176], [581, 176, 182], [589, 176, 187], [596, 178, 187], [614, 180, 184], [627, 183, 181], [636, 185, 179], [647, 190, 176], [662, 202, 163], [669, 208, 157], [670, 208, 155]], [[670, 208, 155], [683, 219, 165], [682, 212, 172], [681, 201, 173], [680, 213, 173], [681, 217, 170], [683, 213, 169], [679, 209, 172], [681, 205, 172], [682, 195, 173], [684, 195, 172], [680, 194, 172], [673, 181, 172], [658, 168, 178], [653, 169, 193], [654, 170, 205], [654, 170, 212], [662, 171, 222], [673, 171, 228], [682, 169, 231], [690, 167, 233], [698, 162, 235], [703, 155, 237], [707, 147, 240], [707, 137, 246], [704, 128, 253], [701, 121, 261], [698, 116, 270], [696, 111, 279], [693, 108, 288], [689, 104, 298], [685, 99, 307], [678, 95, 315], [670, 91, 323], [663, 86, 331], [655, 84, 340], [649, 82, 348], [648, 77, 360], [652, 71, 370], [655, 67, 380], [657, 65, 389], [657, 65, 392]], [[529, 182, 142], [529, 168, 142], [532, 153, 140], [529, 148, 142], [516, 134, 148], [508, 126, 156], [505, 122, 160], [500, 114, 166], [497, 103, 169], [499, 91, 173], [507, 81, 174], [514, 75, 174], [525, 72, 173], [534, 70, 175], [542, 65, 177], [550, 63, 180], [558, 60, 182], [566, 56, 183], [575, 56, 185], [586, 60, 190], [595, 61, 192], [603, 60, 193], [611, 57, 193], [618, 51, 192], [623, 43, 192], [624, 32, 193]], [[678, 210, 174], [681, 196, 175], [679, 187, 176], [676, 180, 175], [669, 170, 175], [658, 162, 176], [648, 157, 175], [633, 156, 178], [623, 154, 190], [623, 164, 206], [628, 183, 215], [633, 191, 218], [637, 198, 222], [644, 204, 223], [651, 208, 223], [659, 213, 224], [667, 219, 229], [674, 221, 232], [681, 224, 237], [687, 226, 242], [693, 225, 249], [693, 224, 260], [691, 223, 269], [686, 219, 277], [681, 217, 287], [676, 220, 299], [672, 223, 309], [667, 228, 316], [662, 232, 322], [657, 234, 327]]]

In [ ]:
spline_points = [[[273, 196, 20], [329, 201, 38], [349, 203, 46], [353, 204, 47], [361, 209, 51], [368, 212, 52], [371, 216, 52], [375, 221, 51], [389, 229, 56], [390, 230, 56], [395, 235, 57], [402, 240, 56], [410, 246, 58], [417, 255, 51], [424, 261, 55], [430, 264, 60], [438, 266, 66], [438, 275, 74], [438, 280, 79], [437, 281, 81], [439, 285, 87], [439, 288, 92], [439, 293, 104], [438, 296, 113], [438, 296, 113]], [[304, 195, 3], [294, 195, 8], [285, 195, 15], [276, 195, 18], [266, 197, 23], [259, 196, 30], [242, 201, 36], [226, 205, 39], [222, 207, 38], [206, 219, 43], [197, 225, 38], [191, 228, 31], [182, 233, 28], [173, 238, 26], [164, 245, 28], [153, 253, 30], [138, 266, 30], [129, 271, 29], [118, 273, 34], [109, 274, 37], [103, 277, 40], [95, 274, 46], [94, 274, 47]], [[188, 230, 30], [196, 217, 34], [207, 202, 49], [218, 187, 60], [212, 185, 71], [203, 183, 79], [191, 181, 83], [177, 170, 86], [170, 165, 93], [171, 169, 95], [169, 168, 93], [169, 162, 96], [164, 156, 102], [159, 151, 112], [154, 146, 120], [150, 140, 128], [145, 127, 136], [132, 120, 143], [124, 116, 146], [126, 113, 154], [127, 107, 164], [125, 100, 172], [127, 92, 180], [130, 84, 187], [134, 76, 193], [136, 73, 201], [135, 76, 211]], [[201, 223, 44], [204, 207, 50], [216, 192, 57], [214, 185, 68], [205, 185, 79], [191, 180, 89], [180, 184, 95], [182, 186, 97], [183, 185, 95], [187, 178, 94], [191, 175, 95], [178, 172, 105], [170, 164, 118], [180, 158, 130], [190, 141, 134], [187, 132, 135], [173, 126, 150], [172, 128, 165], [170, 126, 173]], [[212, 215, 41], [212, 206, 55], [226, 190, 69], [229, 190, 68], [229, 180, 74], [222, 182, 84], [216, 182, 92], [213, 183, 97], [206, 185, 101], [197, 185, 107], [187, 183, 112], [177, 182, 117], [169, 181, 126], [166, 178, 137], [164, 176, 147], [164, 177, 156], [164, 177, 166], [164, 178, 175], [165, 179, 183], [169, 180, 191], [176, 181, 197], [182, 182, 204], [188, 182, 212], [193, 176, 219], [198, 172, 231], [203, 177, 240], [208, 180, 248], [214, 181, 254], [221, 183, 259], [228, 185, 262], [234, 187, 267], [239, 188, 274], [245, 190, 281], [247, 191, 283]], [[256, 197, 33], [260, 188, 46], [269, 173, 52], [257, 172, 65], [256, 175, 65], [247, 181, 73], [241, 190, 83], [230, 196, 94], [227, 198, 98], [221, 205, 104], [214, 210, 110], [207, 211, 120], [203, 218, 130], [193, 225, 133], [190, 235, 144], [189, 238, 152], [189, 244, 159], [188, 248, 166], [189, 255, 173], [191, 260, 180], [195, 265, 186], [198, 269, 192], [202, 274, 197], [205, 279, 203], [206, 288, 206], [210, 295, 211], [212, 303, 216], [212, 311, 221]], [[266, 197, 23], [266, 197, 35], [266, 188, 46], [269, 173, 51], [259, 172, 64], [254, 177, 69], [243, 184, 75], [241, 182, 87], [235, 184, 96], [232, 186, 102], [225, 187, 112], [218, 185, 120], [210, 182, 127], [202, 179, 133], [197, 177, 139], [204, 177, 147], [208, 185, 158], [213, 187, 164], [218, 186, 172], [223, 190, 180], [229, 199, 187], [232, 203, 194], [236, 207, 200], [242, 209, 205], [250, 212, 207], [258, 219, 206], [265, 224, 211], [270, 231, 224], [275, 238, 227], [281, 245, 228], [286, 252, 229], [292, 258, 232], [296, 262, 233]], [[212, 215, 41], [216, 199, 41], [225, 187, 41], [231, 182, 42], [238, 178, 44], [245, 172, 48], [252, 167, 52], [256, 160, 58], [254, 152, 65], [242, 153, 74], [231, 159, 80], [228, 162, 81]], [[212, 215, 41], [210, 215, 53], [199, 229, 54], [192, 238, 57], [190, 244, 52], [188, 252, 43], [185, 259, 36], [180, 265, 26], [172, 272, 20], [162, 277, 16], [153, 282, 13], [142, 288, 13], [132, 293, 14], [122, 298, 18], [114, 304, 24], [107, 308, 31], [100, 314, 38], [93, 318, 47], [89, 322, 56], [87, 326, 65], [83, 331, 73], [81, 336, 82], [81, 342, 90], [81, 346, 98], [81, 351, 105], [82, 357, 111], [84, 363, 117], [88, 368, 124], [91, 373, 129], [94, 377, 136], [95, 383, 142], [97, 386, 150], [98, 385, 156]], [[163, 155, 106], [162, 151, 118], [163, 150, 127], [155, 145, 137], [146, 142, 148], [137, 138, 156], [131, 134, 166], [134, 130, 179], [137, 127, 190], [137, 127, 193]], [[158, 148, 114], [161, 160, 111], [160, 167, 111], [157, 170, 121], [152, 162, 132], [145, 151, 137], [137, 144, 145], [131, 141, 156], [133, 135, 168], [134, 133, 172]], [[165, 179, 186], [167, 183, 198], [170, 186, 204], [173, 189, 210], [175, 193, 218], [177, 195, 227], [178, 196, 235], [179, 197, 243], [180, 198, 251], [181, 200, 259], [180, 204, 267], [181, 208, 276], [190, 204, 284], [197, 202, 294], [199, 202, 303], [201, 204, 311], [209, 209, 317], [217, 212, 322], [219, 213, 323]], [[190, 182, 214], [192, 191, 226], [195, 193, 232], [199, 192, 238], [204, 200, 248], [211, 206, 256], [217, 212, 261], [221, 217, 265], [227, 221, 269], [232, 225, 274], [237, 230, 278], [240, 237, 282], [243, 244, 286], [247, 250, 291], [250, 255, 297], [255, 258, 302], [261, 263, 306], [263, 265, 307]], [[260, 188, 46], [262, 177, 44], [267, 169, 46], [270, 160, 51], [271, 153, 59], [270, 152, 67], [260, 161, 71], [244, 171, 77], [238, 174, 79]], [[191, 232, 140], [193, 242, 139], [193, 249, 142], [194, 255, 146], [194, 262, 151], [196, 269, 156], [198, 274, 163], [200, 279, 169], [201, 281, 171]], [[577, 188, 10], [543, 199, 45], [529, 199, 50], [508, 204, 57], [500, 212, 61], [489, 220, 56], [484, 225, 60], [472, 237, 54], [466, 251, 47], [449, 263, 54], [460, 272, 34], [462, 277, 30], [464, 285, 29]], [[568, 183, 5], [577, 188, 10], [587, 193, 17], [594, 195, 22], [602, 193, 31], [619, 190, 43], [634, 194, 47], [645, 198, 48], [654, 201, 50], [656, 203, 49], [664, 204, 51], [673, 206, 52], [683, 213, 55], [691, 218, 55], [700, 225, 56], [707, 231, 56], [716, 238, 54], [724, 243, 54], [734, 248, 52], [740, 250, 49], [748, 252, 45], [764, 247, 40], [766, 247, 40]], [[543, 199, 45], [551, 199, 58], [562, 188, 69], [568, 183, 73], [575, 181, 74], [583, 179, 75], [592, 181, 75], [600, 187, 78], [605, 194, 80], [608, 202, 84], [612, 208, 88], [616, 212, 94], [620, 217, 99], [624, 220, 106], [626, 222, 115], [625, 224, 123], [623, 227, 132], [618, 232, 139], [611, 239, 145], [601, 242, 150], [591, 246, 156], [588, 248, 158]], [[656, 203, 50], [666, 194, 66], [672, 181, 81], [679, 187, 96], [690, 183, 108], [695, 180, 114], [701, 176, 117], [707, 171, 122], [716, 166, 125], [726, 167, 129], [737, 171, 127], [751, 165, 131], [757, 159, 145], [756, 157, 157], [755, 158, 166], [753, 155, 176], [751, 152, 186], [749, 150, 195], [747, 149, 205], [743, 149, 214], [740, 150, 224], [738, 151, 233], [734, 151, 241], [730, 147, 250], [725, 142, 258], [721, 140, 268], [717, 141, 279], [713, 143, 288], [707, 143, 298], [702, 141, 306], [699, 136, 316], [697, 135, 331], [696, 133, 345], [691, 132, 358], [689, 132, 362]], [[634, 194, 47], [634, 181, 62], [630, 170, 54], [625, 167, 65], [627, 177, 78], [630, 182, 80], [625, 178, 78], [625, 177, 77], [622, 168, 68], [625, 176, 76], [627, 176, 76], [625, 171, 71], [624, 174, 73], [624, 177, 75], [624, 181, 77], [624, 173, 71], [625, 177, 74], [630, 185, 82], [640, 194, 89], [655, 194, 101], [666, 191, 112], [669, 187, 113], [677, 185, 123], [682, 186, 130], [688, 186, 136], [693, 198, 144], [693, 202, 153], [696, 207, 160], [694, 211, 170], [704, 210, 182], [711, 207, 187]], [[679, 210, 54], [677, 197, 67], [676, 187, 77], [679, 186, 85], [686, 187, 85], [694, 185, 95], [700, 184, 101], [706, 180, 107], [709, 177, 116], [708, 170, 125], [715, 165, 136], [720, 162, 142], [724, 159, 149], [727, 158, 158], [729, 158, 166], [731, 158, 174], [730, 159, 183], [726, 160, 191], [719, 164, 199], [710, 165, 207], [703, 164, 216], [699, 163, 227], [704, 163, 240], [695, 171, 253], [696, 174, 265], [696, 174, 273], [685, 174, 282]]]

In [ ]:
spline_points = [[[237, 202, 125], [269, 214, 146], [282, 220, 155], [286, 222, 154], [296, 230, 153], [298, 229, 152], [303, 232, 151], [309, 235, 150], [315, 238, 149], [317, 240, 146], [322, 244, 143], [328, 249, 137], [341, 257, 130], [339, 265, 130], [341, 272, 130], [344, 277, 129], [352, 282, 132], [359, 285, 135], [360, 285, 135]], [[265, 214, 85], [253, 212, 95], [245, 212, 104], [228, 210, 111], [194, 206, 122], [186, 208, 117], [178, 214, 115], [167, 220, 113], [157, 225, 111], [146, 228, 108], [141, 234, 104], [136, 240, 98], [127, 246, 94], [119, 251, 89], [115, 258, 86], [113, 260, 84]], [[164, 222, 112], [157, 207, 114], [158, 200, 124], [163, 198, 130], [167, 197, 142], [170, 198, 149], [169, 201, 155], [169, 203, 164], [165, 207, 172], [155, 210, 178], [144, 212, 182], [134, 210, 187], [121, 209, 190], [110, 209, 195], [99, 210, 200], [89, 204, 203], [80, 195, 206], [72, 187, 210], [69, 178, 220], [74, 174, 232], [80, 181, 241], [85, 186, 250], [86, 187, 259], [85, 186, 268], [87, 185, 278], [89, 184, 287], [90, 184, 295], [92, 181, 303], [96, 178, 312], [96, 180, 322], [97, 181, 332], [98, 181, 340], [102, 178, 348], [103, 177, 350]], [[240, 211, 109], [226, 211, 110], [211, 209, 110], [197, 211, 110], [182, 211, 109], [167, 211, 109], [152, 210, 116], [137, 214, 128], [118, 212, 146], [118, 206, 154], [114, 200, 163], [110, 195, 172], [105, 190, 179], [99, 186, 189], [92, 179, 196], [86, 172, 201], [78, 167, 207], [69, 164, 213], [64, 160, 225], [63, 152, 234], [64, 145, 242], [63, 142, 253], [63, 138, 261], [60, 130, 268], [62, 123, 279], [70, 129, 294]], [[315, 238, 148], [324, 241, 149], [330, 242, 153], [331, 232, 159], [326, 221, 164], [320, 213, 170], [311, 206, 175], [299, 203, 176], [288, 203, 175], [284, 203, 175]], [[228, 210, 111], [228, 210, 126], [237, 205, 141], [234, 200, 152], [229, 204, 158], [220, 209, 163], [215, 214, 166], [211, 222, 168], [205, 229, 171], [199, 236, 175], [192, 242, 177], [189, 252, 181], [188, 257, 188]], [[247, 212, 101], [257, 199, 101], [271, 189, 104], [273, 185, 119], [278, 174, 130], [279, 172, 137], [279, 168, 145], [271, 157, 151], [261, 150, 155], [252, 143, 157], [242, 137, 159], [234, 129, 163], [226, 121, 164], [219, 110, 164], [215, 98, 162], [212, 86, 162], [211, 74, 163], [209, 63, 165], [206, 52, 166], [198, 43, 173], [189, 39, 181], [180, 35, 186], [167, 35, 189], [164, 18, 193], [158, 10, 207], [159, 7, 220], [161, 4, 230]], [[144, 212, 182], [132, 220, 186], [123, 224, 190], [115, 223, 192], [105, 221, 195], [95, 218, 203], [87, 217, 212], [83, 219, 222], [82, 222, 232], [82, 227, 239], [84, 231, 245], [87, 236, 252], [89, 242, 259], [91, 247, 265], [95, 254, 268], [97, 263, 269], [98, 271, 270], [100, 278, 272], [103, 285, 277], [106, 290, 284], [109, 293, 292], [110, 295, 297]], [[89, 184, 293], [93, 190, 305], [94, 192, 308], [98, 196, 315], [101, 199, 322], [106, 200, 329], [110, 201, 336], [109, 204, 345], [110, 204, 354], [120, 208, 361], [124, 211, 368], [126, 212, 377], [135, 213, 383]], [[278, 173, 132], [286, 171, 144], [293, 169, 149], [299, 166, 148], [306, 159, 152], [310, 150, 155], [310, 139, 159], [309, 135, 160]], [[560, 241, 130], [494, 241, 128], [481, 240, 131], [466, 234, 132], [452, 232, 130], [437, 230, 127], [421, 233, 124], [410, 240, 123], [392, 247, 124], [384, 252, 123], [371, 240, 124], [348, 242, 124], [335, 244, 133]], [[475, 220, 55], [482, 220, 66], [489, 220, 77], [494, 220, 85], [502, 224, 93], [509, 223, 99], [511, 228, 113], [513, 232, 123], [526, 234, 122], [534, 237, 125], [549, 241, 129], [562, 243, 131], [574, 246, 131], [582, 248, 127], [591, 253, 127], [600, 256, 127], [611, 258, 129], [619, 263, 124], [628, 268, 122], [635, 272, 119], [642, 278, 115], [661, 283, 99], [663, 284, 97]], [[392, 247, 124], [398, 244, 136], [396, 248, 138], [398, 247, 140], [397, 241, 144], [400, 228, 145], [406, 218, 146], [418, 214, 144], [428, 214, 144], [437, 216, 146], [445, 216, 148], [453, 218, 150], [463, 214, 148], [472, 214, 146], [480, 213, 144], [489, 212, 145], [497, 208, 143], [505, 205, 142], [513, 198, 143]], [[648, 279, 109], [648, 279, 122], [639, 278, 134], [633, 276, 140], [629, 267, 145], [626, 256, 149], [618, 246, 149], [609, 237, 147], [601, 228, 147], [591, 220, 145], [581, 213, 149], [572, 209, 157], [567, 207, 167], [567, 209, 178], [571, 211, 187], [580, 210, 193], [589, 209, 199], [597, 210, 203], [605, 210, 207], [615, 209, 208], [625, 211, 212], [630, 215, 220], [634, 215, 228], [638, 214, 236], [639, 209, 244], [637, 204, 254], [634, 203, 265], [632, 202, 274], [631, 201, 282], [630, 202, 291], [627, 204, 300], [624, 205, 310], [620, 206, 319], [617, 204, 327], [613, 198, 334], [606, 194, 345], [600, 195, 356], [597, 196, 365], [594, 197, 374], [589, 199, 382], [582, 201, 390], [577, 203, 398], [572, 204, 406], [567, 208, 414], [569, 214, 424], [571, 216, 428]], [[635, 272, 119], [635, 272, 134], [626, 264, 145], [625, 258, 148], [619, 247, 150], [611, 238, 147], [601, 230, 147], [594, 220, 150], [593, 209, 158], [598, 206, 174], [601, 204, 182], [606, 197, 189], [612, 192, 194], [617, 189, 201], [626, 186, 205], [632, 182, 209], [640, 181, 214], [646, 179, 220], [651, 177, 226], [657, 174, 231], [661, 170, 239], [660, 161, 247], [657, 153, 254], [654, 148, 265], [656, 141, 273], [652, 139, 286], [648, 136, 295], [645, 135, 305], [646, 130, 315]], [[543, 239, 127], [537, 225, 133], [529, 216, 140], [529, 207, 142], [527, 195, 147], [519, 184, 157], [518, 178, 170], [523, 173, 181], [530, 173, 190], [535, 171, 197], [540, 169, 203], [547, 168, 208], [553, 165, 214], [558, 162, 217]], [[509, 230, 118], [509, 214, 115], [501, 200, 105], [492, 192, 105], [485, 187, 111], [479, 182, 119], [477, 176, 129], [476, 169, 136], [475, 159, 141], [478, 150, 147], [482, 141, 152], [486, 133, 156], [489, 126, 161], [494, 119, 166], [501, 114, 172], [507, 110, 178], [514, 108, 182], [521, 106, 187], [530, 106, 191], [538, 105, 195], [546, 104, 197], [554, 100, 198], [562, 93, 198], [570, 87, 196], [576, 79, 194], [581, 71, 192], [589, 64, 188], [598, 63, 181]], [[499, 223, 90], [499, 210, 90], [487, 195, 83], [476, 184, 72], [470, 181, 70], [459, 175, 73], [450, 166, 73], [444, 154, 72], [438, 144, 74], [437, 133, 83], [439, 128, 94]], [[494, 209, 144], [501, 197, 143], [499, 196, 147], [498, 196, 145], [489, 190, 152], [483, 192, 158], [482, 199, 170], [496, 207, 176], [501, 213, 182], [506, 219, 188], [509, 225, 194], [509, 230, 201], [509, 233, 209], [504, 236, 217]], [[568, 210, 182], [568, 214, 187], [575, 225, 198], [578, 228, 199], [582, 234, 203], [589, 238, 206], [595, 244, 208], [600, 249, 214], [605, 252, 220], [615, 258, 221]], [[624, 205, 310], [621, 213, 321], [618, 219, 328], [615, 221, 336], [612, 223, 345], [610, 228, 353], [610, 233, 360], [608, 238, 367], [607, 239, 370]]]

In [ ]:
spline_points = [[[251, 161, 165], [281, 173, 174], [294, 178, 178], [298, 180, 176], [306, 185, 172], [311, 187, 172], [323, 196, 168], [333, 203, 164], [338, 206, 164], [343, 208, 161], [350, 214, 158], [355, 218, 157], [359, 226, 157], [364, 231, 159], [371, 241, 165], [376, 250, 169], [381, 250, 177], [384, 254, 180], [390, 258, 185], [395, 259, 189], [400, 262, 196], [404, 259, 204], [408, 259, 213], [409, 258, 220], [409, 259, 223]], [[290, 168, 93], [272, 168, 117], [262, 168, 132], [252, 164, 145], [235, 157, 171], [215, 162, 178], [205, 165, 181], [198, 167, 181], [188, 169, 181], [177, 173, 180], [168, 177, 179], [160, 179, 178], [153, 185, 177], [142, 187, 177], [135, 191, 175], [129, 194, 172], [121, 202, 171], [112, 207, 171], [106, 214, 170], [98, 219, 168], [89, 224, 166], [81, 230, 166], [71, 239, 169], [67, 241, 170]], [[89, 231, 167], [92, 218, 168], [102, 203, 166], [103, 203, 167], [115, 189, 175], [119, 180, 179], [121, 173, 184], [122, 175, 186], [123, 175, 183], [122, 177, 186], [124, 173, 192], [131, 165, 197], [132, 155, 202], [129, 150, 201], [122, 129, 210], [119, 125, 216], [116, 119, 224], [110, 112, 230], [100, 108, 236], [91, 104, 242], [83, 96, 246], [76, 87, 249], [69, 80, 257], [66, 68, 262]], [[281, 173, 174], [278, 158, 183], [268, 146, 186], [263, 140, 188], [255, 134, 186], [246, 126, 183], [234, 120, 183], [222, 118, 187], [211, 117, 194], [202, 117, 201], [192, 118, 206], [184, 123, 218], [177, 128, 228], [168, 131, 235], [159, 133, 240], [150, 134, 245], [141, 138, 249], [133, 140, 258], [129, 142, 268], [129, 144, 279], [131, 146, 287], [132, 150, 295], [136, 154, 301], [141, 158, 306], [145, 162, 311], [150, 167, 315], [157, 171, 319], [164, 174, 324], [173, 177, 326], [181, 181, 329], [189, 185, 331], [195, 190, 334], [201, 194, 336], [206, 201, 338], [208, 203, 339]], [[357, 223, 156], [360, 212, 156], [369, 204, 155], [376, 204, 157], [384, 207, 165], [387, 207, 173]], [[129, 194, 172], [139, 192, 157], [143, 192, 145], [146, 192, 138], [143, 198, 130], [141, 205, 123], [137, 212, 118], [131, 220, 114], [124, 229, 115], [117, 239, 117], [111, 247, 119], [104, 254, 123], [99, 261, 127], [91, 267, 133], [82, 270, 140], [72, 271, 145], [61, 272, 150], [45, 271, 161], [32, 271, 170], [24, 270, 177], [21, 269, 180]], [[137, 189, 176], [130, 184, 191], [133, 179, 192], [135, 175, 190], [138, 162, 192], [140, 153, 197], [142, 145, 200], [141, 134, 204], [141, 126, 208], [143, 125, 219], [138, 124, 229], [132, 127, 238], [126, 132, 246], [117, 134, 252], [108, 136, 257], [98, 137, 263], [89, 140, 271], [85, 141, 283], [83, 144, 292], [80, 149, 301], [79, 152, 310], [81, 156, 318], [85, 160, 326], [90, 165, 333], [93, 168, 340], [96, 173, 348], [99, 175, 353]], [[207, 164, 181], [214, 164, 166], [211, 164, 158], [200, 158, 158], [193, 150, 165], [195, 153, 162], [201, 159, 159], [204, 162, 159], [192, 153, 165], [192, 150, 168], [195, 154, 164], [191, 153, 167], [191, 149, 172], [185, 145, 182], [180, 136, 194], [173, 130, 201], [165, 130, 202], [152, 132, 205], [143, 130, 214], [139, 125, 226], [134, 124, 235], [123, 123, 241], [113, 119, 246], [103, 117, 251], [94, 115, 258], [87, 110, 266], [83, 104, 274], [79, 99, 284], [77, 95, 294], [76, 91, 305], [74, 88, 315], [73, 85, 325], [74, 83, 335], [77, 80, 343], [81, 78, 352], [85, 78, 361], [89, 77, 369], [92, 75, 377], [96, 74, 385], [99, 71, 393], [105, 67, 400], [109, 63, 407], [113, 60, 415], [120, 59, 424], [124, 59, 429]], [[252, 164, 145], [253, 152, 142], [255, 149, 127], [256, 137, 131], [253, 136, 135], [252, 136, 134], [255, 137, 134], [257, 137, 135], [252, 138, 135], [257, 139, 129], [255, 142, 128], [260, 133, 124], [259, 125, 124], [260, 119, 126], [264, 113, 138], [265, 106, 145], [264, 99, 155], [263, 91, 162], [261, 84, 171], [258, 77, 178], [256, 70, 185], [250, 62, 191], [242, 55, 195], [233, 49, 201], [225, 43, 204], [216, 37, 206], [210, 26, 209], [201, 18, 210], [193, 11, 213], [185, 2, 214]], [[264, 168, 128], [263, 156, 128], [257, 142, 126], [257, 139, 131], [256, 131, 143], [255, 126, 150], [254, 120, 156], [253, 111, 161], [247, 102, 169], [241, 98, 178], [235, 94, 186], [228, 90, 193], [223, 86, 206], [219, 84, 216], [212, 82, 225], [205, 78, 232], [197, 74, 239], [187, 70, 244], [183, 69, 246]], [[272, 168, 117], [279, 154, 105], [290, 143, 100], [289, 134, 106], [292, 126, 108], [294, 118, 109], [298, 108, 109], [301, 97, 110], [299, 87, 117], [298, 81, 124]], [[192, 118, 206], [192, 107, 207], [193, 98, 211], [191, 94, 220], [182, 90, 231], [172, 86, 235], [163, 81, 239], [157, 76, 242]], [[136, 139, 254], [136, 131, 262], [133, 127, 270], [133, 128, 278], [136, 130, 290], [143, 132, 302], [150, 134, 308], [155, 135, 311]], [[164, 174, 324], [165, 179, 336], [167, 182, 342], [171, 186, 346], [176, 190, 351], [181, 194, 356], [188, 199, 360], [194, 203, 364], [201, 205, 366], [208, 205, 375], [213, 203, 383], [218, 202, 390], [223, 201, 397], [228, 202, 404], [230, 202, 406]], [[73, 85, 325], [80, 85, 334], [85, 84, 341], [88, 83, 348], [94, 90, 355], [100, 97, 361], [102, 98, 363]], [[96, 73, 388], [97, 77, 399], [95, 76, 403], [94, 78, 414], [99, 72, 416], [105, 67, 426]], [[222, 41, 205], [212, 38, 206], [204, 33, 206], [195, 29, 207], [184, 24, 208], [173, 21, 214], [166, 18, 223], [161, 14, 232], [156, 10, 242], [148, 6, 250], [145, 4, 252]], [[294, 118, 109], [288, 106, 112], [283, 100, 118], [284, 95, 128], [288, 91, 137], [292, 87, 144], [294, 86, 146]], [[729, 239, 149], [737, 211, 141], [741, 201, 139], [743, 193, 139], [751, 180, 141], [752, 177, 142], [755, 172, 145], [756, 166, 148], [754, 168, 156], [756, 162, 163], [760, 158, 166], [760, 156, 166]], [[528, 146, 159], [489, 155, 185], [475, 159, 186], [463, 157, 183], [452, 159, 182], [438, 165, 175], [424, 180, 172], [414, 182, 168], [413, 184, 171]], [[470, 156, 103], [493, 152, 126], [505, 149, 140], [510, 146, 149], [513, 151, 165], [529, 148, 165], [539, 146, 164], [545, 144, 163], [552, 151, 166], [558, 151, 164], [569, 154, 164], [578, 155, 162], [585, 159, 164], [594, 162, 164], [617, 174, 161], [636, 186, 157], [644, 193, 153], [670, 214, 150], [685, 224, 148], [694, 228, 147], [705, 234, 146], [717, 230, 145], [728, 237, 149], [736, 242, 154], [745, 241, 160]], [[470, 158, 184], [457, 153, 198], [453, 140, 201], [457, 130, 202], [460, 125, 203], [466, 120, 203], [476, 117, 202], [484, 116, 201], [492, 116, 200], [501, 118, 200], [508, 128, 198], [519, 133, 189], [528, 131, 183], [536, 126, 179], [547, 119, 182], [552, 115, 195], [554, 114, 205], [557, 115, 213], [563, 116, 221], [571, 117, 225], [579, 118, 229], [587, 119, 234], [592, 121, 239], [598, 123, 245], [602, 127, 252], [604, 131, 260], [602, 133, 268], [598, 135, 277], [594, 137, 287], [590, 141, 295], [586, 145, 303], [582, 149, 311], [576, 154, 318], [571, 159, 327], [570, 161, 330]], [[470, 158, 184], [484, 144, 195], [491, 134, 188], [497, 127, 187], [505, 119, 186], [518, 117, 182], [529, 120, 179], [540, 124, 176], [551, 126, 174], [563, 123, 177], [573, 113, 184], [583, 108, 190], [592, 105, 195], [604, 104, 200], [613, 101, 208], [624, 99, 214], [629, 96, 224], [631, 90, 236], [639, 81, 246], [641, 75, 257], [639, 71, 270], [638, 69, 281], [636, 69, 292], [631, 70, 303], [626, 70, 314], [621, 71, 325], [614, 72, 337], [606, 71, 346]], [[625, 179, 159], [625, 168, 167], [618, 157, 179], [613, 150, 184], [605, 143, 184], [595, 135, 182], [587, 127, 183], [584, 114, 187], [591, 104, 198], [603, 102, 204], [612, 100, 208], [620, 100, 212], [628, 99, 218], [635, 101, 224], [644, 102, 229], [652, 100, 233], [660, 96, 236], [666, 91, 243], [668, 86, 251], [670, 81, 261], [674, 77, 270], [676, 74, 279], [678, 72, 288], [680, 68, 296], [677, 65, 305], [673, 65, 315], [671, 65, 324], [670, 65, 332], [669, 65, 340], [667, 64, 348], [664, 64, 358], [663, 65, 367], [662, 65, 375], [659, 65, 383], [656, 65, 391], [653, 66, 399], [649, 68, 407], [644, 71, 415], [641, 75, 424], [641, 76, 427]], [[510, 146, 149], [504, 140, 133], [492, 125, 122], [484, 119, 123], [485, 114, 127], [484, 104, 139], [484, 95, 147], [484, 89, 153], [484, 81, 159], [486, 72, 166], [488, 63, 170], [492, 53, 174], [496, 45, 181], [500, 36, 185], [504, 28, 189], [509, 21, 191], [516, 13, 194], [522, 7, 196], [529, 2, 198], [531, 1, 198]], [[493, 152, 126], [480, 150, 119], [465, 135, 106], [453, 123, 103], [446, 114, 101], [442, 105, 102], [438, 94, 103], [434, 83, 104], [433, 71, 110], [433, 67, 113]], [[501, 118, 200], [508, 108, 199], [515, 103, 197], [521, 103, 192], [530, 104, 188], [540, 111, 192], [545, 117, 201], [547, 122, 208], [551, 127, 215], [557, 131, 220], [562, 135, 226], [567, 139, 232], [571, 143, 237], [573, 144, 239]], [[547, 119, 182], [555, 126, 173], [560, 126, 172], [570, 125, 181], [573, 109, 188], [573, 102, 199], [573, 91, 203], [577, 82, 212], [581, 77, 218]], [[591, 104, 198], [603, 98, 198], [605, 89, 204], [610, 86, 209], [617, 82, 212], [625, 80, 216], [633, 77, 220], [645, 71, 224], [652, 62, 228], [653, 59, 229]], [[628, 99, 218], [640, 93, 219], [648, 85, 223], [653, 82, 226], [661, 78, 230], [668, 73, 235], [675, 68, 240], [680, 62, 246], [686, 58, 252], [691, 52, 258], [694, 44, 264], [693, 38, 276], [690, 35, 285], [689, 29, 294], [690, 27, 305], [692, 28, 313]], [[652, 66, 402], [645, 57, 406], [643, 53, 412], [637, 49, 422], [626, 48, 429], [621, 47, 430]], [[484, 88, 155], [493, 85, 166], [495, 85, 169], [503, 83, 174], [509, 78, 179], [511, 67, 184], [522, 60, 190], [525, 52, 195]], [[494, 50, 176], [504, 45, 182], [510, 43, 185], [515, 36, 189], [520, 29, 194], [526, 26, 200], [531, 22, 207], [533, 21, 209]], [[573, 102, 199], [580, 100, 211], [583, 101, 213], [590, 101, 219], [597, 100, 223], [605, 100, 227], [612, 100, 230], [620, 96, 233]], [[689, 26, 301], [689, 18, 309], [689, 15, 315], [682, 11, 325], [682, 7, 336], [681, 2, 344], [683, 3, 356], [680, 1, 363]]]

In [ ]:
spline_points_total = []
for n in range(len(spline_points)):
    spline_points_total.append(spline_points[n])

In [ ]:
#8个训练样本
lcs = []#亮度系数集
ccs = []#标准差系数集
bcs = []#偏转系数集
dcs = []#密度系数集
pcs = []#目标点周围亮点数
nlcs = []#非先验点亮度系数集
nccs = []#非先验点标准差系数集
nbcs = []#非先验点偏转系数集
ndcs = []#非先验点密度系数集
npcs = []#非目标点周围亮点数

chang,kuan,gao = target_3d_insert_clear.shape

for train in range(8):
    lcs.append([])# = [[],[],[],[],[],[],[],[]]#亮度系数集
    ccs.append([])# = [[],[],[],[],[],[],[],[]]#标准差系数集
    bcs.append([])# = [[],[],[],[],[],[],[],[]]#偏转系数集
    dcs.append([])# = [[],[],[],[],[],[],[],[]]#密度系数集
    pcs.append([])# = [[],[],[],[],[],[],[],[]]#目标点周围亮点数
    nlcs.append([])# = [[],[],[],[],[],[],[],[]]#非先验点亮度系数集
    nccs.append([])# = [[],[],[],[],[],[],[],[]]#非先验点标准差系数集
    nbcs.append([])# = [[],[],[],[],[],[],[],[]]#非先验点偏转系数集
    ndcs.append([])# = [[],[],[],[],[],[],[],[]]#非先验点密度系数集
    npcs.append([])# = [[],[],[],[],[],[],[],[]]#非目标点周围亮点数
    
    #锥形搜索
    #cita是锥形中心沿z轴偏转角度
    #omiga是中心线周围点沿中心线偏转角度
    #gama是锥形中心线沿x轴偏转角度,为逆时针旋转
    #sigma是中心线周围点沿初始x轴偏转角度基础上继续偏转的角度

    real_center_points = []
    compare_points = []

    x1 = float(given_points[train][0][0])
    y1 = float(given_points[train][0][1])
    z1 = float(given_points[train][0][2])
    print(x1,y1,z1)
    stoppoint = [x1,y1,z1]
    real_center_point = stoppoint
    real_center_points.append(real_center_point)

    x2 = float(given_points[train][1][0])
    y2 = float(given_points[train][1][1])
    z2 = float(given_points[train][1][2])

    startpoint = [x1,y1,z1]
    stoppoint = [x2,y2,z2]
    realcenterpoint = stoppoint
    real_center_points.append(realcenterpoint)


    if z2-z1 == 0 and x2-x1 >= 0:
        cita = float(math.pi*0.5)
    elif z2-z1 == 0 and x2-x1 < 0:
        cita = float(math.pi*0.5)
    elif (x2-x1)**2+(y2-y1)**2 == 0:
        cita = float(0)
    elif z2-z1 < 0:
        cita = math.pi-float(math.atan((math.sqrt((x2-x1)**2+(y2-y1)**2))/(math.sqrt((z2-z1)**2))))
    else:
        cita = float(math.atan((math.sqrt((x2-x1)**2+(y2-y1)**2))/(math.sqrt((z2-z1)**2))))
    alpha = cita
    #print(cita)

    if x2-x1 == 0 and y2-y1 > 0:
        gama = float(math.pi*0.5)
    elif x2-x1 == 0 and y2-y1 < 0:
        gama = float(-math.pi*0.5)
    elif x2-x1 == 0 and y2-y1 == 0:
        gama = float(0)
    elif x2-x1 < 0 and y2-y1 < 0:
        gama = float(math.atan((y2-y1)/(x2-x1)))+math.pi
    elif x2-x1 < 0 and y2-y1 == 0:
        gama = math.pi
    else:
        gama = float(math.atan((y2-y1)/(x2-x1)))
    beta = gama
    #print(gama)

    d = 3

    omiga1 = 0
    omiga2 = float((20/180)*math.pi)
    omiga3 = float((45/180)*math.pi)
    omiga4 = float((45/180)*math.pi)
    omiga5 = float((40/180)*math.pi)
    omiga6 = float((50/180)*math.pi)
    #对应第1周偏转点
    sigma1 = float((0/180)*math.pi)
    sigma2 = float((45/180)*math.pi)
    sigma3 = float((90/180)*math.pi)
    sigma4 = float((135/180)*math.pi)
    sigma5 = float((180/180)*math.pi)
    sigma6 = float((225/180)*math.pi)
    sigma7 = float((270/180)*math.pi)
    sigma8 = float((315/180)*math.pi)
    #对应第2周偏转点
    sigma9 = float((24/180)*math.pi)
    sigma10 = float((48/180)*math.pi)
    sigma11 = float((72/180)*math.pi)
    sigma12 = float((96/180)*math.pi)
    sigma13 = float((120/180)*math.pi)
    sigma14 = float((144/180)*math.pi)
    sigma15 = float((168/180)*math.pi)
    sigma16 = float((192/180)*math.pi)
    sigma17 = float((216/180)*math.pi)
    sigma18 = float((240/180)*math.pi)
    sigma19 = float((264/180)*math.pi)
    sigma20 = float((288/180)*math.pi)
    sigma21 = float((312/180)*math.pi)
    sigma22 = float((336/180)*math.pi)
    sigma23 = float((0/180)*math.pi)
    #对应第3周偏转点
    sigma24 = float((15/180)*math.pi)
    sigma25 = float((30/180)*math.pi)
    sigma26 = float((45/180)*math.pi)
    sigma27 = float((60/180)*math.pi)
    sigma28 = float((75/180)*math.pi)
    sigma29 = float((90/180)*math.pi)
    sigma30 = float((105/180)*math.pi)
    sigma31 = float((120/180)*math.pi)
    sigma32 = float((135/180)*math.pi)
    sigma33 = float((150/180)*math.pi)
    sigma34 = float((165/180)*math.pi)
    sigma35 = float((180/180)*math.pi)
    sigma36 = float((195/180)*math.pi)
    sigma37 = float((210/180)*math.pi)
    sigma38 = float((225/180)*math.pi)
    sigma39 = float((240/180)*math.pi)
    sigma40 = float((255/180)*math.pi)
    sigma41 = float((270/180)*math.pi)
    sigma42 = float((285/180)*math.pi)
    sigma43 = float((300/180)*math.pi)
    sigma44 = float((315/180)*math.pi)
    sigma45 = float((330/180)*math.pi)
    sigma46 = float((345/180)*math.pi)
    #对应第4周偏转点
    sigma47 = float((0*11.25/180)*math.pi)
    sigma48 = float((1*11.25/180)*math.pi)
    sigma49 = float((2*11.25/180)*math.pi)
    sigma50 = float((3*11.25/180)*math.pi)
    sigma51 = float((4*11.25/180)*math.pi)
    sigma52 = float((5*11.25/180)*math.pi)
    sigma53 = float((6*11.25/180)*math.pi)
    sigma54 = float((7*11.25/180)*math.pi)
    sigma55 = float((8*11.25/180)*math.pi)
    sigma56 = float((9*11.25/180)*math.pi)
    sigma57 = float((10*11.25/180)*math.pi)
    sigma58 = float((11*11.25/180)*math.pi)
    sigma59 = float((12*11.25/180)*math.pi)
    sigma60 = float((13*11.25/180)*math.pi)
    sigma61 = float((14*11.25/180)*math.pi)
    sigma62 = float((15*11.25/180)*math.pi)
    sigma63 = float((16*11.25/180)*math.pi)
    sigma64 = float((17*11.25/180)*math.pi)
    sigma65 = float((18*11.25/180)*math.pi)
    sigma66 = float((19*11.25/180)*math.pi)
    sigma67 = float((20*11.25/180)*math.pi)
    sigma68 = float((21*11.25/180)*math.pi)
    sigma69 = float((22*11.25/180)*math.pi)
    sigma70 = float((23*11.25/180)*math.pi)
    sigma71 = float((24*11.25/180)*math.pi)
    sigma72 = float((25*11.25/180)*math.pi)
    sigma73 = float((26*11.25/180)*math.pi)
    sigma74 = float((27*11.25/180)*math.pi)
    sigma75 = float((28*11.25/180)*math.pi)
    sigma76 = float((29*11.25/180)*math.pi)
    sigma77 = float((30*11.25/180)*math.pi)
    sigma78 = float((31*11.25/180)*math.pi)
    #对应第5周偏转点
    sigma117 = float((1*9/180)*math.pi)
    sigma79 = float((2*9/180)*math.pi)
    sigma80 = float((3*9/180)*math.pi)
    sigma81 = float((4*9/180)*math.pi)
    sigma82 = float((5*9/180)*math.pi)
    sigma83 = float((6*9/180)*math.pi)
    sigma84 = float((7*9/180)*math.pi)
    sigma85 = float((8*9/180)*math.pi)
    sigma86 = float((9*9/180)*math.pi)
    sigma87 = float((10*9/180)*math.pi)
    sigma88 = float((11*9/180)*math.pi)
    sigma89 = float((12*9/180)*math.pi)
    sigma90 = float((13*9/180)*math.pi)
    sigma91 = float((14*9/180)*math.pi)
    sigma92 = float((15*9/180)*math.pi)
    sigma93 = float((16*9/180)*math.pi)
    sigma94 = float((17*9/180)*math.pi)
    sigma95 = float((18*9/180)*math.pi)
    sigma96 = float((19*9/180)*math.pi)
    sigma97 = float((20*9/180)*math.pi)
    sigma98 = float((21*9/180)*math.pi)
    sigma99 = float((22*9/180)*math.pi)
    sigma100 = float((23*9/180)*math.pi)
    sigma101 = float((24*9/180)*math.pi)
    sigma102 = float((25*9/180)*math.pi)
    sigma103 = float((26*9/180)*math.pi)
    sigma104 = float((27*9/180)*math.pi)
    sigma105 = float((28*9/180)*math.pi)
    sigma106 = float((29*9/180)*math.pi)
    sigma107 = float((30*9/180)*math.pi)
    sigma108 = float((31*9/180)*math.pi)
    sigma109 = float((32*9/180)*math.pi)
    sigma110 = float((33*9/180)*math.pi)
    sigma111 = float((34*9/180)*math.pi)
    sigma112 = float((35*9/180)*math.pi)
    sigma113 = float((36*9/180)*math.pi)
    sigma114 = float((37*9/180)*math.pi)
    sigma115 = float((38*9/180)*math.pi)
    sigma116 = float((39*9/180)*math.pi)
    #锥形搜索矩阵
    zhuixing_matrix = [[(d*math.sin(omiga1)*math.cos(sigma1)*math.cos(cita)+d*math.cos(omiga1)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga1)*math.sin(sigma1)*math.sin(gama), (d*math.sin(omiga1)*math.cos(sigma1)*math.cos(cita)+d*math.cos(omiga1)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga1)*math.sin(sigma1)*math.cos(gama), -d*math.sin(omiga1)*math.cos(sigma1)*math.sin(cita)+d*math.cos(omiga1)*math.cos(cita)],
                      [(d*math.sin(omiga2)*math.cos(sigma1)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga2)*math.sin(sigma1)*math.sin(gama), (d*math.sin(omiga2)*math.cos(sigma1)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga2)*math.sin(sigma1)*math.cos(gama), -d*math.sin(omiga2)*math.cos(sigma1)*math.sin(cita)+d*math.cos(omiga2)*math.cos(cita)],
                      [(d*math.sin(omiga2)*math.cos(sigma2)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga2)*math.sin(sigma2)*math.sin(gama), (d*math.sin(omiga2)*math.cos(sigma2)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga2)*math.sin(sigma2)*math.cos(gama), -d*math.sin(omiga2)*math.cos(sigma2)*math.sin(cita)+d*math.cos(omiga2)*math.cos(cita)],
                      [(d*math.sin(omiga2)*math.cos(sigma3)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga2)*math.sin(sigma3)*math.sin(gama), (d*math.sin(omiga2)*math.cos(sigma3)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga2)*math.sin(sigma3)*math.cos(gama), -d*math.sin(omiga2)*math.cos(sigma3)*math.sin(cita)+d*math.cos(omiga2)*math.cos(cita)],
                      [(d*math.sin(omiga2)*math.cos(sigma4)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga2)*math.sin(sigma4)*math.sin(gama), (d*math.sin(omiga2)*math.cos(sigma4)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga2)*math.sin(sigma4)*math.cos(gama), -d*math.sin(omiga2)*math.cos(sigma4)*math.sin(cita)+d*math.cos(omiga2)*math.cos(cita)],
                      [(d*math.sin(omiga2)*math.cos(sigma5)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga2)*math.sin(sigma5)*math.sin(gama), (d*math.sin(omiga2)*math.cos(sigma5)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga2)*math.sin(sigma5)*math.cos(gama), -d*math.sin(omiga2)*math.cos(sigma5)*math.sin(cita)+d*math.cos(omiga2)*math.cos(cita)],
                      [(d*math.sin(omiga2)*math.cos(sigma6)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga2)*math.sin(sigma6)*math.sin(gama), (d*math.sin(omiga2)*math.cos(sigma6)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga2)*math.sin(sigma6)*math.cos(gama), -d*math.sin(omiga2)*math.cos(sigma6)*math.sin(cita)+d*math.cos(omiga2)*math.cos(cita)],
                      [(d*math.sin(omiga2)*math.cos(sigma7)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga2)*math.sin(sigma7)*math.sin(gama), (d*math.sin(omiga2)*math.cos(sigma7)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga2)*math.sin(sigma7)*math.cos(gama), -d*math.sin(omiga2)*math.cos(sigma7)*math.sin(cita)+d*math.cos(omiga2)*math.cos(cita)],
                      [(d*math.sin(omiga2)*math.cos(sigma8)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga2)*math.sin(sigma8)*math.sin(gama), (d*math.sin(omiga2)*math.cos(sigma8)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga2)*math.sin(sigma8)*math.cos(gama), -d*math.sin(omiga2)*math.cos(sigma8)*math.sin(cita)+d*math.cos(omiga2)*math.cos(cita)],
                      [(d*math.sin(omiga3)*math.cos(sigma9)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma9)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma9)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma9)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma9)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                      [(d*math.sin(omiga3)*math.cos(sigma10)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma10)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma10)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma10)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma10)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                      [(d*math.sin(omiga3)*math.cos(sigma11)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma11)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma11)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma11)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma11)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                      [(d*math.sin(omiga3)*math.cos(sigma12)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma12)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma12)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma12)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma12)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                      [(d*math.sin(omiga3)*math.cos(sigma13)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma13)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma13)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma13)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma13)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                      [(d*math.sin(omiga3)*math.cos(sigma14)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma14)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma14)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma14)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma14)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                      [(d*math.sin(omiga3)*math.cos(sigma15)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma15)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma15)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma15)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma15)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                      [(d*math.sin(omiga3)*math.cos(sigma16)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma16)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma16)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma16)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma16)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                      [(d*math.sin(omiga3)*math.cos(sigma17)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma17)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma17)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma17)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma17)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                      [(d*math.sin(omiga3)*math.cos(sigma18)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma18)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma18)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma18)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma18)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                      [(d*math.sin(omiga3)*math.cos(sigma19)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma19)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma19)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma19)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma19)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                      [(d*math.sin(omiga3)*math.cos(sigma20)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma20)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma20)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma20)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma20)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                      [(d*math.sin(omiga3)*math.cos(sigma21)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma21)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma21)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma21)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma21)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                      [(d*math.sin(omiga3)*math.cos(sigma22)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma22)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma22)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma22)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma22)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                      [(d*math.sin(omiga3)*math.cos(sigma23)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma23)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma23)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma23)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma23)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)]]
    #开始迭代
    for repeat in range(120):
        #print('迭代次数')
        #print(repeat)

        bias = 0
        brightness_sum = []
        cross_var_sum = []
        points = []
        #print(d)
        if d == 3:
            startpoint = stoppoint
        else:
            startpoint = startpoint

        x1 = startpoint[0]
        y1 = startpoint[1]
        z1 = startpoint[2]
        #print(x1,y1,z1)

        for r1 in range(20):
            if target_3d_insert_clear_1[round(x1-r1*math.cos(cita)*math.cos(gama)),round(y1-r1*math.cos(cita)*math.sin(gama)),round(z1+r1*math.sin(cita))] == 0:
                break

        for r2 in range(20):
            if target_3d_insert_clear_1[round(x1+r2*math.cos(cita)*math.cos(gama)),round(y1+r2*math.cos(cita)*math.sin(gama)),round(z1-r2*math.sin(cita))] == 0:
                break

        r = int((r1+r2)/2)
        #print(r,d)

        #第一次搜索
        for i,j,k in zhuixing_matrix:

            #print(i,j,k)
            x2 = round(x1+i)
            y2 = round(y1+j)
            z2 = round(z1+k)
            #print(x2,y2,z2)

            if target_3d_insert_clear_1[x2,y2,z2] == 0:
                pass
            else:
                if z2-z1 == 0:
                    alpha = 0.5*math.pi
                elif z2-z1 < 0:
                    alpha = math.pi-math.atan(math.sqrt((x2-x1)**2+(y2-y1)**2)/abs(z2-z1))
                else:
                    alpha = math.atan(math.sqrt((x2-x1)**2+(y2-y1)**2)/(z2-z1))

                
                if x2-x1 == 0 and y2-y1 > 0:
                    beta = 0.5*math.pi
                elif x2-x1 == 0 and y2-y1 < 0:
                    beta = 1.5*math.pi
                elif x2-x1 >= 0 and y2-y1 == 0:
                    beta = float(0)
                elif y2-y1 == 0 and x2-x1 < 0:
                    beta = math.pi*(1)
                elif y2-y1>0 and x2-x1<0:
                    beta = math.pi*1-math.atan((abs(y2-y1)/abs(x2-x1)))
                elif y2-y1<0 and x2-x1<0:
                    beta = math.atan((abs(y2-y1)/abs(x2-x1)))+math.pi*1
                elif x2-x1>0 and y2-y1<0:
                    beta = math.atan((y2-y1)/(x2-x1))+math.pi*2
                else:
                    beta = math.atan((y2-y1)/(x2-x1))

                #print(alpha,beta)

                point = 0
                for l,m,n in d3_plat_1:  
                    if target_3d_insert_clear_1[x2+l,y2+m,z2+n] == 255:
                        point = point+1
                points.append(point)
                #print(points)
                
                #计算圆形程度，先算4个方向的距离，最后求方差
                for cross1 in range(30):
                    if target_3d_insert_clear_1[round(x1-cross1*math.cos(alpha)*math.cos(beta)),round(y1-cross1*math.cos(alpha)*math.sin(beta)),round(z1+cross1*math.sin(alpha))] == 0:
                        break
                for cross2 in range(30):
                    if target_3d_insert_clear_1[round(x1+cross2*math.cos(alpha)*math.cos(beta)),round(y1+cross2*math.cos(alpha)*math.sin(beta)),round(z1-cross2*math.sin(alpha))] == 0:
                        break
                for cross3 in range(30):
                    if target_3d_insert_clear_1[round(x1-cross3*math.sin(beta)),round(y1+cross3*math.cos(beta)),round(z1+0)] == 0:
                        break
                for cross4 in range(30):
                    if target_3d_insert_clear_1[round(x1+cross4*math.sin(beta)),round(y1-cross4*math.cos(beta)),round(z1+0)] == 0:
                        break

                cross_average = (cross1+cross2+cross3+cross4)*0.25

                #cross_var = math.sqrt(((cross1-cross_average)**2+(cross2-cross_average)**2+(cross3-cross_average)**2+(cross4-cross_average)**2)/4)
                cross_var = abs(cross1-cross_average)+abs(cross2-cross_average)+abs(cross3-cross_average)+abs(cross4-cross_average)
                if cross_var == 0:
                    cross_var = 0.01
                #print(cross_var_sum)
                cross_var_sum.append(cross_var)
                #print(cross_var)

                #算各个方向圆柱亮度和
                brightness = 0
                repeat_points = []
                for a in range(-r*2,r*2+1):
                    for b in range(-r*2,r*2+1):
                        for c in range(-r*2,r*2+1):
                            #print(round(math.cos(-beta)*math.cos(-alpha)*(a) - math.sin(-beta)*math.cos(-alpha)*(b)+math.sin(-alpha)*(c)),round(math.sin(-beta)*(a) + math.cos(-beta)*(b)),round((-1)*math.sin(-alpha)*math.cos(-beta)*(a)+math.sin(-alpha)*math.sin(-beta)*(b)+math.cos(-alpha)*(c)))
                            if round(math.cos(-beta)*math.cos(-alpha)*(a) - math.sin(-beta)*math.cos(-alpha)*(b)+math.sin(-alpha)*(c))**2 + round(math.sin(-beta)*(a) + math.cos(-beta)*(b))**2 <= r**2:
                                if 0 <= round((-1)*math.sin(-alpha)*math.cos(-beta)*(a)+math.sin(-alpha)*math.sin(-beta)*(b)+math.cos(-alpha)*(c)) <= d:
                                    if [round(math.cos(-beta)*math.cos(-alpha)*(a) - math.sin(-beta)*math.cos(-alpha)*(b)+math.sin(-alpha)*(c)),round(math.sin(-beta)*(a) + math.cos(-beta)*(b)),round((-1)*math.sin(-alpha)*math.cos(-beta)*(a)+math.sin(-alpha)*math.sin(-beta)*(b)+math.cos(-alpha)*(c))] not in repeat_points:
                                        if target_3d_insert_clear_1[int(x1+a),int(y1+b),int(z1+c)] == 255:
                                            brightness = brightness + 1
                                            repeat_point = [round(math.cos(-beta)*math.cos(-alpha)*(a) - math.sin(-beta)*math.cos(-alpha)*(b)+math.sin(-alpha)*(c)),round(math.sin(-beta)*(a) + math.cos(-beta)*(b)),round((-1)*math.sin(-alpha)*math.cos(-beta)*(a)+math.sin(-alpha)*math.sin(-beta)*(b)+math.cos(-alpha)*(c))]
                                            repeat_points.append(repeat_point)
                                            #if (-1)*math.sin(-alpha)*math.cos(-beta)*(a)+math.sin(-alpha)*math.sin(-beta)*(b)+math.cos(-alpha)*(c) == 0:
                                            #print((math.cos(-beta)*math.cos(-alpha)*(a) - math.sin(-beta)*math.cos(-alpha)*(b)+math.sin(-alpha)*(c)),math.sin(-beta)*(a) + math.cos(-beta)*(b),(-1)*math.sin(-alpha)*math.cos(-beta)*(a)+math.sin(-alpha)*math.sin(-beta)*(b)+math.cos(-alpha)*(c))

                brightness_sum.append(brightness)
                #print(brightness)
        #print(max(brightness_sum),min(cross_var_sum))
        #print(len(brightness_sum))
        if len(brightness_sum) == 0:
            d = d+1
            if d <= 7:
                zhuixing_matrix = [[(d*math.sin(omiga1)*math.cos(sigma1)*math.cos(cita)+d*math.cos(omiga1)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga1)*math.sin(sigma1)*math.sin(gama), (d*math.sin(omiga1)*math.cos(sigma1)*math.cos(cita)+d*math.cos(omiga1)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga1)*math.sin(sigma1)*math.cos(gama), -d*math.sin(omiga1)*math.cos(sigma1)*math.sin(cita)+d*math.cos(omiga1)*math.cos(cita)],
                                  [(d*math.sin(omiga2)*math.cos(sigma1)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga2)*math.sin(sigma1)*math.sin(gama), (d*math.sin(omiga2)*math.cos(sigma1)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga2)*math.sin(sigma1)*math.cos(gama), -d*math.sin(omiga2)*math.cos(sigma1)*math.sin(cita)+d*math.cos(omiga2)*math.cos(cita)],
                                  [(d*math.sin(omiga2)*math.cos(sigma2)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga2)*math.sin(sigma2)*math.sin(gama), (d*math.sin(omiga2)*math.cos(sigma2)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga2)*math.sin(sigma2)*math.cos(gama), -d*math.sin(omiga2)*math.cos(sigma2)*math.sin(cita)+d*math.cos(omiga2)*math.cos(cita)],
                                  [(d*math.sin(omiga2)*math.cos(sigma3)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga2)*math.sin(sigma3)*math.sin(gama), (d*math.sin(omiga2)*math.cos(sigma3)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga2)*math.sin(sigma3)*math.cos(gama), -d*math.sin(omiga2)*math.cos(sigma3)*math.sin(cita)+d*math.cos(omiga2)*math.cos(cita)],
                                  [(d*math.sin(omiga2)*math.cos(sigma4)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga2)*math.sin(sigma4)*math.sin(gama), (d*math.sin(omiga2)*math.cos(sigma4)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga2)*math.sin(sigma4)*math.cos(gama), -d*math.sin(omiga2)*math.cos(sigma4)*math.sin(cita)+d*math.cos(omiga2)*math.cos(cita)],
                                  [(d*math.sin(omiga2)*math.cos(sigma5)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga2)*math.sin(sigma5)*math.sin(gama), (d*math.sin(omiga2)*math.cos(sigma5)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga2)*math.sin(sigma5)*math.cos(gama), -d*math.sin(omiga2)*math.cos(sigma5)*math.sin(cita)+d*math.cos(omiga2)*math.cos(cita)],
                                  [(d*math.sin(omiga2)*math.cos(sigma6)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga2)*math.sin(sigma6)*math.sin(gama), (d*math.sin(omiga2)*math.cos(sigma6)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga2)*math.sin(sigma6)*math.cos(gama), -d*math.sin(omiga2)*math.cos(sigma6)*math.sin(cita)+d*math.cos(omiga2)*math.cos(cita)],
                                  [(d*math.sin(omiga2)*math.cos(sigma7)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga2)*math.sin(sigma7)*math.sin(gama), (d*math.sin(omiga2)*math.cos(sigma7)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga2)*math.sin(sigma7)*math.cos(gama), -d*math.sin(omiga2)*math.cos(sigma7)*math.sin(cita)+d*math.cos(omiga2)*math.cos(cita)],
                                  [(d*math.sin(omiga2)*math.cos(sigma8)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga2)*math.sin(sigma8)*math.sin(gama), (d*math.sin(omiga2)*math.cos(sigma8)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga2)*math.sin(sigma8)*math.cos(gama), -d*math.sin(omiga2)*math.cos(sigma8)*math.sin(cita)+d*math.cos(omiga2)*math.cos(cita)],
                                  [(d*math.sin(omiga3)*math.cos(sigma9)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma9)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma9)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma9)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma9)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                                  [(d*math.sin(omiga3)*math.cos(sigma10)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma10)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma10)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma10)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma10)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                                  [(d*math.sin(omiga3)*math.cos(sigma11)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma11)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma11)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma11)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma11)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                                  [(d*math.sin(omiga3)*math.cos(sigma12)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma12)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma12)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma12)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma12)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                                  [(d*math.sin(omiga3)*math.cos(sigma13)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma13)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma13)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma13)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma13)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                                  [(d*math.sin(omiga3)*math.cos(sigma14)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma14)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma14)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma14)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma14)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                                  [(d*math.sin(omiga3)*math.cos(sigma15)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma15)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma15)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma15)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma15)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                                  [(d*math.sin(omiga3)*math.cos(sigma16)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma16)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma16)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma16)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma16)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                                  [(d*math.sin(omiga3)*math.cos(sigma17)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma17)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma17)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma17)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma17)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                                  [(d*math.sin(omiga3)*math.cos(sigma18)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma18)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma18)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma18)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma18)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                                  [(d*math.sin(omiga3)*math.cos(sigma19)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma19)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma19)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma19)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma19)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                                  [(d*math.sin(omiga3)*math.cos(sigma20)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma20)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma20)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma20)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma20)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                                  [(d*math.sin(omiga3)*math.cos(sigma21)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma21)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma21)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma21)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma21)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                                  [(d*math.sin(omiga3)*math.cos(sigma22)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma22)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma22)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma22)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma22)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                                  [(d*math.sin(omiga3)*math.cos(sigma23)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma23)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma23)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma23)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma23)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)]]                   
                
            else:
                break
        else:
            #第二次搜索
            for i,j,k in zhuixing_matrix:
                
                x2 = round(x1+i)
                y2 = round(y1+j)
                z2 = round(z1+k)

                bias = bias + 1

                if target_3d_insert_clear_1[x2,y2,z2] == 0:
                    pass
                else:
                    if z2-z1 == 0:
                        alpha = 0.5*math.pi
                    elif z2-z1 < 0:
                        alpha = math.pi-math.atan(math.sqrt((x2-x1)**2+(y2-y1)**2)/abs(z2-z1))
                    else:
                        alpha = math.atan(math.sqrt((x2-x1)**2+(y2-y1)**2)/(z2-z1))

                    if x2-x1 == 0 and y2-y1 > 0:
                        beta = 0.5*math.pi
                    elif x2-x1 == 0 and y2-y1 < 0:
                        beta = 1.5*math.pi
                    elif x2-x1 == 0 and y2-y1 == 0:
                        beta = float(0)
                    elif y2-y1 == 0 and x2-x1 < 0:
                        beta = math.pi*(-1)
                    elif y2-y1>0 and x2-x1<0:
                        beta = math.pi*1-math.atan((abs(y2-y1)/abs(x2-x1)))
                    elif y2-y1<0 and x2-x1<0:
                        beta = math.atan((abs(y2-y1)/abs(x2-x1)))+math.pi*1
                    elif x2-x1>0 and y2-y1<0:
                        beta = math.atan((y2-y1)/(x2-x1))+math.pi*2
                    else:
                        beta = math.atan((y2-y1)/(x2-x1))
                        
                    point = 0
                    for l,m,n in d3_plat_1:  
                        if target_3d_insert_clear_1[x2+l,y2+m,z2+n] == 255:
                            point = point + 1
                    
                    pc = 1
                    nppoints = np.array(points)
                    nppoints2 = nppoints.copy()
                    points2 = nppoints2.tolist()
                    
                    for e in range(20):
                        if point == max(points2):
                            pc = pc
                            break
                        else:
                            points2.remove(max(points2))
                            pc = pc+1

                    #再一次计算圆形程度，先算4个方向的距离，最后求方差
                    for cross1 in range(30):
                        if target_3d_insert_clear_1[round(x1-cross1*math.cos(alpha)*math.cos(beta)),round(y1-cross1*math.cos(alpha)*math.sin(beta)),round(z1+cross1*math.sin(alpha))] == 0:
                            break
                    for cross2 in range(30):
                        if target_3d_insert_clear_1[round(x1+cross2*math.cos(alpha)*math.cos(beta)),round(y1+cross2*math.cos(alpha)*math.sin(beta)),round(z1-cross2*math.sin(alpha))] == 0:
                            break
                    for cross3 in range(30):
                        if target_3d_insert_clear_1[round(x1-cross3*math.sin(beta)),round(y1+cross3*math.cos(beta)),round(z1+0)] == 0:
                            break
                    for cross4 in range(30):
                        if target_3d_insert_clear_1[round(x1+cross4*math.sin(beta)),round(y1-cross4*math.cos(beta)),round(z1+0)] == 0:
                            break

                    cross_average = (cross1+cross2+cross3+cross4)*0.25
                    #cross_var = math.sqrt(((cross1-cross_average)**2+(cross2-cross_average)**2+(cross3-cross_average)**2+(cross4-cross_average)**2)/4)
                    cross_var = abs(cross1-cross_average)+abs(cross2-cross_average)+abs(cross3-cross_average)+abs(cross4-cross_average)
                    if cross_var == 0:
                        cross_var = 0.01

                    #再算一次亮度和           
                    brightness = 0
                    repeat_points = []
                    for a in range(round(-r*2),round(r*2+1)):
                        for b in range(round(-r*2),round(r*2+1)):
                            for c in range(round(-r*2),round(r*2+1)):
                                if round(math.cos(-beta)*math.cos(-alpha)*(a) - math.sin(-beta)*math.cos(-alpha)*(b)+math.sin(-alpha)*(c))**2 + round(math.sin(-beta)*(a) + math.cos(-beta)*(b))**2 <= r**2:
                                    if 0 <= round((-1)*math.sin(-alpha)*math.cos(-beta)*(a)+math.sin(-alpha)*math.sin(-beta)*(b)+math.cos(-alpha)*(c)) <= d:
                                        if [round(math.cos(-beta)*math.cos(-alpha)*(a) - math.sin(-beta)*math.cos(-alpha)*(b)+math.sin(-alpha)*(c)),round(math.sin(-beta)*(a) + math.cos(-beta)*(b)),round((-1)*math.sin(-alpha)*math.cos(-beta)*(a)+math.sin(-alpha)*math.sin(-beta)*(b)+math.cos(-alpha)*(c))] not in repeat_points:
                                            if target_3d_insert_clear_1[int(x1+a),int(y1+b),int(z1+c)] == 255:
                                                brightness = brightness + 1
                                                repeat_point = [round(math.cos(-beta)*math.cos(-alpha)*(a) - math.sin(-beta)*math.cos(-alpha)*(b)+math.sin(-alpha)*(c)),round(math.sin(-beta)*(a) + math.cos(-beta)*(b)),round((-1)*math.sin(-alpha)*math.cos(-beta)*(a)+math.sin(-alpha)*math.sin(-beta)*(b)+math.cos(-alpha)*(c))]
                                                repeat_points.append(repeat_point)

                    if bias == 1:
                        bc = 1
                    elif 1 < bias <= 9:
                        bc = 2
                    elif 9 < bias <= 25:
                        bc = 3
                    scale_dc = 5
                    if scale_dc<x2<chang and scale_dc<y2<kuan and scale_dc<z2<gao:
                        array = target_3d_insert_same[x2-scale_dc:x2+scale_dc,y2-scale_dc:y2+scale_dc,z2-scale_dc:z2+scale_dc]#target_3d_insert_clear[x2-scale_dc:x2+scale_dc,y2-scale_dc:y2+scale_dc,z2-scale_dc:z2+scale_dc]
                        result = compute_3d_hessian_matrix(array, sigma=1.5, scale=True, whiteonblack=True)
                        A = np.mat(result[scale_dc][scale_dc][scale_dc])
                        U = A*A.T
                        lamda,hU=np.linalg.eig(U)
                        sigma=np.sqrt(lamda)
                        result2 = [sigma[0],sigma[1],sigma[2]]
                        result2.sort()
                        lamda0 = result2[0]
                        lamda1 = result2[1]
                        lamda2 = result2[2]
                        dc = np.var(result2)
                        #print(dc)
#                         if lamda2/lamda0>3: #and lamda2/lamda0>5 and (lamda1/lamda0<3 or lamda1-lamda0<3):
#                             dc = 1
#                         else:
#                             dc = 0
                    else:
                        dc = 0
                    if scale_dc<x2<chang and scale_dc<y2<kuan and scale_dc<z2<gao:
                        array = target_3d_insert[x2-scale_dc:x2+scale_dc,y2-scale_dc:y2+scale_dc,z2-scale_dc:z2+scale_dc]#target_3d_insert_clear[x2-scale_dc:x2+scale_dc,y2-scale_dc:y2+scale_dc,z2-scale_dc:z2+scale_dc]
                        result = compute_3d_hessian_matrix(array, sigma=1.5, scale=True, whiteonblack=True)
                        A = np.mat(result[scale_dc][scale_dc][scale_dc])
                        U = A*A.T
                        lamda,hU=np.linalg.eig(U)
                        sigma=np.sqrt(lamda)
                        result2 = [sigma[0],sigma[1],sigma[2]]
                        result2.sort()
                        lamda0 = result2[0]
                        lamda1 = result2[1]
                        lamda2 = result2[2]
                        dc1 = np.var(result2)
                        #print(dc)
#                         if lamda2/lamda0>3: #and lamda2/lamda0>5 and (lamda1/lamda0<3 or lamda1-lamda0<3):
#                             dc = 1
#                         else:
#                             dc = 0
                    else:
                        dc1 = 0

                    #if max(brightness_sum)*(0.76)<= brightness <= max(brightness_sum)*(0.92):
                        #if min(cross_var_sum)*(1) <= cross_var <= min(cross_var_sum)*(2):
                            #print(brightness/max(brightness_sum),cross_var/min(cross_var_sum))
                    #print(max(points))

                    if [x2,y2,z2] not in given_points[train]:
                        #print([x2,y2,z2])
                        print('no',dc)
                        nlc = brightness/max(brightness_sum)
                        ncc = cross_var/min(cross_var_sum)
                        nlcs[train].append(nlc)
                        nccs[train].append(ncc)
                        nbcs[train].append(bc)
                        ndcs[train].append(dc+dc1)
                        npcs[train].append(pc)

                    else:
                        #print([x2,y2,z2])
                        #print(point)
                        #print(alpha,beta)
                        print('yes',dc)
                        lc = brightness/max(brightness_sum)
                        cc = cross_var/min(cross_var_sum)
                        lcs[train].append(lc)
                        ccs[train].append(cc)
                        bcs[train].append(bc)
                        dcs[train].append(dc)
                        pcs[train].append(pc)
                        

                        #确定下一次锥形步长
                        d = 3

                        stoppoint = [x2,y2,z2]
                        #real_center_point = [x2,y2,z2]
                        #compare_point = [round(stoppoint[0]/insert),round(stoppoint[1]/insert),round(stoppoint[2]/insert)]#用于在512*640*248的图上做对比，插值后的图像micron看不了
                        #real_center_points.append(real_center_point)
                        #compare_points.append(compare_point)

                        cita = alpha
                        gama = beta

                        #重新定义锥形搜索矩阵
                        #锥形搜索矩阵
                        zhuixing_matrix = [[(d*math.sin(omiga1)*math.cos(sigma1)*math.cos(cita)+d*math.cos(omiga1)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga1)*math.sin(sigma1)*math.sin(gama), (d*math.sin(omiga1)*math.cos(sigma1)*math.cos(cita)+d*math.cos(omiga1)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga1)*math.sin(sigma1)*math.cos(gama), -d*math.sin(omiga1)*math.cos(sigma1)*math.sin(cita)+d*math.cos(omiga1)*math.cos(cita)],
                                          [(d*math.sin(omiga2)*math.cos(sigma1)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga2)*math.sin(sigma1)*math.sin(gama), (d*math.sin(omiga2)*math.cos(sigma1)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga2)*math.sin(sigma1)*math.cos(gama), -d*math.sin(omiga2)*math.cos(sigma1)*math.sin(cita)+d*math.cos(omiga2)*math.cos(cita)],
                                          [(d*math.sin(omiga2)*math.cos(sigma2)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga2)*math.sin(sigma2)*math.sin(gama), (d*math.sin(omiga2)*math.cos(sigma2)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga2)*math.sin(sigma2)*math.cos(gama), -d*math.sin(omiga2)*math.cos(sigma2)*math.sin(cita)+d*math.cos(omiga2)*math.cos(cita)],
                                          [(d*math.sin(omiga2)*math.cos(sigma3)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga2)*math.sin(sigma3)*math.sin(gama), (d*math.sin(omiga2)*math.cos(sigma3)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga2)*math.sin(sigma3)*math.cos(gama), -d*math.sin(omiga2)*math.cos(sigma3)*math.sin(cita)+d*math.cos(omiga2)*math.cos(cita)],
                                          [(d*math.sin(omiga2)*math.cos(sigma4)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga2)*math.sin(sigma4)*math.sin(gama), (d*math.sin(omiga2)*math.cos(sigma4)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga2)*math.sin(sigma4)*math.cos(gama), -d*math.sin(omiga2)*math.cos(sigma4)*math.sin(cita)+d*math.cos(omiga2)*math.cos(cita)],
                                          [(d*math.sin(omiga2)*math.cos(sigma5)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga2)*math.sin(sigma5)*math.sin(gama), (d*math.sin(omiga2)*math.cos(sigma5)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga2)*math.sin(sigma5)*math.cos(gama), -d*math.sin(omiga2)*math.cos(sigma5)*math.sin(cita)+d*math.cos(omiga2)*math.cos(cita)],
                                          [(d*math.sin(omiga2)*math.cos(sigma6)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga2)*math.sin(sigma6)*math.sin(gama), (d*math.sin(omiga2)*math.cos(sigma6)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga2)*math.sin(sigma6)*math.cos(gama), -d*math.sin(omiga2)*math.cos(sigma6)*math.sin(cita)+d*math.cos(omiga2)*math.cos(cita)],
                                          [(d*math.sin(omiga2)*math.cos(sigma7)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga2)*math.sin(sigma7)*math.sin(gama), (d*math.sin(omiga2)*math.cos(sigma7)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga2)*math.sin(sigma7)*math.cos(gama), -d*math.sin(omiga2)*math.cos(sigma7)*math.sin(cita)+d*math.cos(omiga2)*math.cos(cita)],
                                          [(d*math.sin(omiga2)*math.cos(sigma8)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga2)*math.sin(sigma8)*math.sin(gama), (d*math.sin(omiga2)*math.cos(sigma8)*math.cos(cita)+d*math.cos(omiga2)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga2)*math.sin(sigma8)*math.cos(gama), -d*math.sin(omiga2)*math.cos(sigma8)*math.sin(cita)+d*math.cos(omiga2)*math.cos(cita)],
                                          [(d*math.sin(omiga3)*math.cos(sigma9)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma9)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma9)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma9)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma9)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                                          [(d*math.sin(omiga3)*math.cos(sigma10)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma10)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma10)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma10)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma10)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                                          [(d*math.sin(omiga3)*math.cos(sigma11)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma11)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma11)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma11)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma11)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                                          [(d*math.sin(omiga3)*math.cos(sigma12)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma12)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma12)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma12)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma12)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                                          [(d*math.sin(omiga3)*math.cos(sigma13)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma13)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma13)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma13)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma13)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                                          [(d*math.sin(omiga3)*math.cos(sigma14)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma14)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma14)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma14)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma14)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                                          [(d*math.sin(omiga3)*math.cos(sigma15)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma15)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma15)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma15)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma15)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                                          [(d*math.sin(omiga3)*math.cos(sigma16)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma16)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma16)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma16)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma16)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                                          [(d*math.sin(omiga3)*math.cos(sigma17)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma17)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma17)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma17)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma17)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                                          [(d*math.sin(omiga3)*math.cos(sigma18)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma18)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma18)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma18)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma18)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                                          [(d*math.sin(omiga3)*math.cos(sigma19)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma19)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma19)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma19)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma19)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                                          [(d*math.sin(omiga3)*math.cos(sigma20)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma20)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma20)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma20)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma20)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                                          [(d*math.sin(omiga3)*math.cos(sigma21)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma21)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma21)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma21)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma21)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                                          [(d*math.sin(omiga3)*math.cos(sigma22)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma22)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma22)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma22)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma22)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)],
                                          [(d*math.sin(omiga3)*math.cos(sigma23)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.cos(gama)-d*math.sin(omiga3)*math.sin(sigma23)*math.sin(gama), (d*math.sin(omiga3)*math.cos(sigma23)*math.cos(cita)+d*math.cos(omiga3)*math.sin(cita))*math.sin(gama)+d*math.sin(omiga3)*math.sin(sigma23)*math.cos(gama), -d*math.sin(omiga3)*math.cos(sigma23)*math.sin(cita)+d*math.cos(omiga3)*math.cos(cita)]]
                        #print(repeat + 1)
                        break


In [ ]:
contents_yes = []
contents_no = []
classify = []
for train in range(8):
    for n in range(len(lcs[train])):
        content_yes = [lcs[train][n],ccs[train][n],bcs[train][n],dcs[train][n],pcs[train][n]]
        contents_yes.append(content_yes)
    
for train in range(8):
    for n in range(len(nlcs[train])):
        content_no = [nlcs[train][n],nccs[train][n],nbcs[train][n],ndcs[train][n],npcs[train][n]]
        contents_no.append(content_no)

contents = contents_yes + contents_no
for n in range(len(contents_yes)):
    classify.append([1])
for n in range(len(contents_no)):
    classify.append([2])

npcontents = np.array(contents)
npclassify = np.array(classify)

In [ ]:
contents

In [ ]:
classify

In [ ]:
accuracy = [contents,classify]
file = open(give_your_path,'w')
file.write(str(accuracy))
file.close()

In [ ]:
npclassify.shape

In [ ]:
npcontents.shape

In [ ]:
np.version.version

In [ ]:
import pandas

In [ ]:
train_contents = np.hstack((contents,classify))

data_df = pd.DataFrame(train_contents)
data_df.to_excel(excel_path)

In [ ]:
def excel2m(path):#读excel数据转为矩阵函数
    data = xlrd.open_workbook(path)
    table = data.sheets()[0] # 获取excel中第一个sheet表
    nrows = table.nrows  # 行数
    ncols = table.ncols  # 列数
    datamatrix = np.zeros((nrows, ncols))
    for x in range(ncols):
        cols = table.col_values(x)
        cols1 = np.matrix(cols)  # 把list转换为矩阵进行矩阵操作
        datamatrix[:, x] = cols1 # 把数据进行存储
    return datamatrix

In [ ]:
book = excel2m(your_path)
book.shape

In [ ]:
contents,classifys = np.hsplit(book,[5])
print(classifys)

In [ ]:
clf_rf2 = RandomForestClassifier(n_estimators = 2000)
clf_rf2.fit(npcontents[:2000],npclassify[:2000])
rfc_s2 = cross_val_score(clf_rf2, npcontents,npclassify,cv=5)
print(rfc_s2)

In [ ]:
#mask
from sklearn.model_selection import cross_val_score
rfc_s = cross_val_score(clf_rf, npcontents,npclassify,cv=5)
print(rfc_s)

In [ ]:
with open(r'E:\article3\clf_rf_20230227.pickle', 'wb') as fw:
    pickle.dump(clf_rf2, fw)

In [ ]:
clf_rf.feature_importances_